<a href="https://colab.research.google.com/github/basharbme/3D-Pelvic-Bone-cancer-classification/blob/main/Copy_of_Pelvic_bone_3d_classification_Part_2_evaluation_claassification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [ ]:
import logging
import os
import sys

import numpy as np
import torch
from torch.utils.data import DataLoader

import monai
from monai.data import CSVSaver
from monai.transforms import AddChanneld, Compose, LoadImaged, Resized, ScaleIntensityd, EnsureTyped


def main():
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)

    
    data_path = os.sep.join(["/content/drive/My Drive/Data_folder/"])
    images = [
        
        "test_2.nii"
    ]
    images = [os.sep.join([data_path, f]) for f in images]

    # 2 binary labels for tumour classification: metastasis and sarcoma
    labels = np.array([1, 0], dtype=np.int64)
    val_files = [{"img": img, "label": label} for img, label in zip(images, labels)]

    # Define transforms for image
    val_transforms = Compose(
        [
            LoadImaged(keys=["img"]),
            AddChanneld(keys=["img"]),
            ScaleIntensityd(keys=["img"]),
            Resized(keys=["img"], spatial_size=(96, 96, 96)),
            EnsureTyped(keys=["img"]),
        ]
    )

    # create a validation data loader
    val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=2, num_workers=4, pin_memory=torch.cuda.is_available())

    # Create DenseNet121
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)

    model.load_state_dict(torch.load("/content/drive/My Drive/best_metric_model_classification3d_dict.pth"))
    model.eval()
    with torch.no_grad():
        num_correct = 0.0
        metric_count = 0
        saver = CSVSaver(output_dir="./output")
        for val_data in val_loader:
            val_images, val_labels = val_data["img"].to(device), val_data["label"].to(device)
            val_outputs = model(val_images).argmax(dim=1)
            value = torch.eq(val_outputs, val_labels)
            metric_count += len(value)
            num_correct += value.sum().item()
            saver.save_batch(val_outputs, val_data["img_meta_dict"])
        metric = num_correct / metric_count
        print("evaluation metric:", metric)
        saver.finalize()


if __name__ == "__main__":
    main()

MONAI version: 0.7.0
Numpy version: 1.19.5
Pytorch version: 1.9.0+cu111
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: bfa054b9c3064628a21f4c35bbe3132964e91f43

Optional dependencies:
Pytorch Ignite version: 0.4.5
Nibabel version: 3.0.2
scikit-image version: 0.16.2
Pillow version: 7.1.2
Tensorboard version: 2.6.0
gdown version: 3.6.4
TorchVision version: 0.10.0+cu111
tqdm version: 4.62.3
lmdb version: 0.99
psutil version: 5.4.8
pandas version: 1.1.5
einops version: 0.3.2
transformers version: 4.11.3

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.


evaluation metric: 1.0


In [ ]:
!pip install monai-deploy-app-sdk

     |████████████████████████████████| 119 kB 5.3 MB/s 
  Attempting uninstall: typeguard
    Found existing installation: typeguard 2.7.1
    Uninstalling typeguard-2.7.1:
      Successfully uninstalled typeguard-2.7.1


In [ ]:
!pip install monailabel

     |████████████████████████████████| 6.2 MB 4.8 MB/s 
     |████████████████████████████████| 74 kB 2.8 MB/s 
     |████████████████████████████████| 2.7 MB 26.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 36.7 MB/s 
     |████████████████████████████████| 51 kB 6.3 MB/s 
     |████████████████████████████████| 10.1 MB 22.8 MB/s 
     |████████████████████████████████| 1.6 MB 42.8 MB/s 
     |████████████████████████████████| 650 kB 36.9 MB/s 
     |████████████████████████████████| 1.9 MB 41.7 MB/s 
     |████████████████████████████████| 46 kB 3.2 MB/s 
     |████████████████████████████████| 76 kB 5.6 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 60 kB 7.4 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
     |████████████████████████████████| 221 kB 46.1 MB/s 
     |████████████

In [ ]:
!pip install SimpleITK==2.1.0
!pip install torchsummaryX
!pip install nibabel
!pip install pillow
!pip install tensorboard
!pip install gdown
!pip install pytorch-ignite==0.4.4
!pip install itk
!pip install tqdm
!pip install lmdb
!pip install psutil
!pip install pandas
!pip install einops
!pip install scikit-image
!pip install 'monai[all]'

     |████████████████████████████████| 48.4 MB 40 kB/s 
  Attempting uninstall: SimpleITK
    Found existing installation: SimpleITK 2.1.1
    Uninstalling SimpleITK-2.1.1:
      Successfully uninstalled SimpleITK-2.1.1
     |████████████████████████████████| 200 kB 5.2 MB/s 
  Attempting uninstall: pytorch-ignite
    Found existing installation: pytorch-ignite 0.4.5
    Uninstalling pytorch-ignite-0.4.5:
      Successfully uninstalled pytorch-ignite-0.4.5
     |████████████████████████████████| 4.9 MB 5.4 MB/s 
     |████████████████████████████████| 2.9 MB 35.3 MB/s 
  Using cached pytorch_ignite-0.4.5-py3-none-any.whl (221 kB)
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 50.6 MB/s 
     |████████████████████████████████| 895 kB 43.4 MB/s 
  Attempting uninstall: pytorch-ignite
    Found existing installation: pytorch-ignite 0.4.4
    Uninstalling pytorch-ignite-0.4.4:
      Successfully uninstalled pytorch-ignite-0.4.4
